# Results of All Models

Elif Cansu YILDIZ  
Salih Marangoz

In [ ]:
# For non-deterministic inference
%env CUBLAS_WORKSPACE_CONFIG=4096:8

In [1]:
%load_ext autoreload
%autoreload 2

# Download IO Routines
#!wget "https://lmb.informatik.uni-freiburg.de/resources/datasets/IO.py" -nv -O io_routines.py

import os
import sys
from PIL import Image
import numpy as np
from torchvision import transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.tensorboard import SummaryWriter

from models.psmnet import PSMNet, FeatureExtractionPlusResNet50
from models.gcnet import GCnet
from models.psmnet_basic import PSMNet as PSMNet_basic

import dataset_utilities as d_utils
import training_utilities as t_utils

In [2]:
print("Module versions:")
print('\n'.join(f'{"> " + m.__name__}: {m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))
plt.style.use('seaborn')

Module versions:
> PIL.Image: 8.3.1
> numpy: 1.20.3
> torch: 1.10.0.dev20210814
> torchvision: 0.11.0.dev20210814


In [4]:
DATASET_ROOT = "../../dataset"
driving_path = os.path.join(DATASET_ROOT, "driving")
monkaa_path = os.path.join(DATASET_ROOT, "monkaa")
flyingthings3d_path = os.path.join(DATASET_ROOT, "flyingthings3d")
kitti_path = os.path.join(DATASET_ROOT, "kitti")

In [5]:
pretrain_transforms = transforms.Compose([
    d_utils.ToTensorMulti(),
    d_utils.RandomCropMulti((256,512)),
    d_utils.NormalizeMulti(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

finetune_transforms = transforms.Compose([
    d_utils.ToTensorMulti(),
    d_utils.RandomCropMulti((256,512)),
    d_utils.ColorJitterMulti(brightness=0.3, hue=.05),
    d_utils.NormalizeMulti(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

eval_transforms = transforms.Compose([
    d_utils.ToTensorMulti(),
    d_utils.SanitizeImageSizesMulti(),
    d_utils.NormalizeMulti(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [6]:
driving_dataset = d_utils.DrivingDataset(driving_path, transforms=pretrain_transforms)
monkaa_dataset = d_utils.MonkaaDataset(monkaa_path, transforms=pretrain_transforms)
flyingthings3d_dataset = d_utils.Flyingthings3dDataset(flyingthings3d_path, transforms=pretrain_transforms)
kitti_train_val, kitti_eval = d_utils.KittiDataset(kitti_path, train_transforms=finetune_transforms, eval_transforms=eval_transforms).split_dataset()

In [7]:
train1, val1 = d_utils.split_dataset(driving_dataset, first_part=0.95, second_part=0.05)
train2, val2 = d_utils.split_dataset(monkaa_dataset, first_part=0.95, second_part=0.05)
train3, val3 = d_utils.split_dataset(flyingthings3d_dataset, first_part=0.95, second_part=0.05)

finetuning_train_dataset, finetuning_val_dataset = d_utils.split_dataset(kitti_train_val, 
                                                                         first_part=0.8, 
                                                                         second_part=0.2)

pretraining_train_dataset = torch.utils.data.ConcatDataset((train1, train2, train3))
pretraining_val_dataset = torch.utils.data.ConcatDataset((val1, val2, val3))

print("Size of the finetuning_train_dataset:", len(finetuning_train_dataset))
print("Size of the finetuning_val_dataset:", len(finetuning_val_dataset))
print("Size of the pretraining_train_dataset:", len(pretraining_train_dataset))
print("Size of the pretraining_val_dataset:", len(pretraining_val_dataset))

Size of the finetuning_train_dataset: 120
Size of the finetuning_val_dataset: 30
Size of the pretraining_train_dataset: 35172
Size of the pretraining_val_dataset: 1852


In [8]:
train_batch_size = 1
eval_batch_size = 1

finetuning_train_dataloader = d_utils.make_dataloader(finetuning_train_dataset, batch_size=train_batch_size)
finetuning_val_dataloader = d_utils.make_dataloader(finetuning_val_dataset, batch_size=eval_batch_size)
pretraining_train_dataloader = d_utils.make_dataloader(pretraining_train_dataset, batch_size=train_batch_size)
pretraining_val_dataloader = d_utils.make_dataloader(pretraining_val_dataset, batch_size=eval_batch_size)
test_dataloader = d_utils.make_dataloader(kitti_eval, batch_size=1) # needs to be 1 since image sizes can differ

In [9]:
device = t_utils.get_device()
print(f"Device is : {device}")
print(torch.cuda.get_device_name(torch.cuda.current_device()))

Device is : cuda
NVIDIA GeForce RTX 3080 Laptop GPU


## Pretrained Models

### Full PSMNet

**With default cost volume**

In [10]:
model = PSMNet(192).to(device)
e1 = t_utils.Experiment(model        = model,
                        criterion    = nn.SmoothL1Loss(), 
                        scheduler    = None,
                        optimizer    = torch.optim.Adam(model.parameters(), lr=1e-4), 
                        train_loader = pretraining_train_dataloader, 
                        val_loader   = pretraining_val_dataloader, 
                        device       = device)

e1.load("runs/PSM-pretraining-2021_09_22-08_31_27_192disp_10epoch_default/model_manual_save.pt")

output = e1.eval_model(test_dataloader)
print("Mean 3PE: {}%, Mean SmoothL1 loss: {}".format(round((1-output["accuracy"])*100,2), 
                                                     round(output["loss"], 4)))

Mean 3PE: 14.37%, Mean SmoothL1 loss: 1.801900029182434


**With strided cost volume**

In [11]:
costvolume_slide = 192//2
costvolume_stride = 2
model = PSMNet(192, costvolume_slide=costvolume_slide, costvolume_stride=costvolume_stride).to(device)
e1 = t_utils.Experiment(model        = model,
                        criterion    = nn.SmoothL1Loss(), 
                        scheduler    = None,
                        optimizer    = torch.optim.Adam(model.parameters(), lr=1e-4), 
                        train_loader = pretraining_train_dataloader, 
                        val_loader   = pretraining_val_dataloader, 
                        device       = device)

e1.load("runs/PSM-pretraining-2021_09_22-14_34_14_192disp_10epoch_costslide2stride2/model_manual_save.pt")

output = e1.eval_model(test_dataloader)
print("Mean 3PE: {}%, Mean SmoothL1 loss: {}".format(round((1-output["accuracy"])*100,2), 
                                                     round(output["loss"], 4)))

Mean 3PE: 14.92%, Mean SmoothL1 loss: 1.6742000579833984


**With ResNet50 module concatenated to the fusion layer of feature extraction part**

In [12]:
model = PSMNet(192).to(device)
model.feature_extraction = FeatureExtractionPlusResNet50().to(device)

e1 = t_utils.Experiment(model        = model,
                        criterion    = nn.SmoothL1Loss(), 
                        scheduler    = None,
                        optimizer    = torch.optim.Adam(model.parameters(), lr=1e-4), 
                        train_loader = pretraining_train_dataloader, 
                        val_loader   = pretraining_val_dataloader, 
                        device       = device)
e1.load("runs/PSM-Resnet50-pretraining-2021_09_26-07_47_52/model_manual_save.pt")

output = e1.eval_model(test_dataloader)
print("Mean 3PE: {}%, Mean SmoothL1 loss: {}".format(round((1-output["accuracy"])*100,2), 
                                                     round(output["loss"], 4)))

Downloading: "https://download.pytorch.org/models/fcn_resnet50_coco-1167a1af.pth" to /home/cansu/.cache/torch/hub/checkpoints/fcn_resnet50_coco-1167a1af.pth


  0%|          | 0.00/135M [00:00<?, ?B/s]

Mean 3PE: 12.13%, Mean SmoothL1 loss: 1.4759000539779663


### GCNet

In [13]:
model = GCnet(192).to(device)

e1 = t_utils.Experiment(model        = model,
                        criterion    = nn.SmoothL1Loss(), 
                        scheduler    = None,
                        optimizer    = torch.optim.Adam(model.parameters(), lr=1e-4), 
                        train_loader = pretraining_train_dataloader, 
                        val_loader   = pretraining_val_dataloader, 
                        device       = device)

e1.load("runs/GCnet-pretraining-2021_09_24-16_50_30/model_manual_save.pt")

output = e1.eval_model(test_dataloader)
print("Mean 3PE: {}%, Mean SmoothL1 loss: {}".format(round((1-output["accuracy"])*100,2), 
                                                     round(output["loss"], 4)))

Mean 3PE: 10.34%, Mean SmoothL1 loss: 1.454699993133545


### Basic PSMNet

In [14]:
model = PSMNet_basic(192).to(device)

e1 = t_utils.Experiment(model        = model,
                        criterion    = nn.SmoothL1Loss(), 
                        scheduler    = None,
                        optimizer    = torch.optim.Adam(model.parameters(), lr=1e-4), 
                        train_loader = pretraining_train_dataloader, 
                        val_loader   = pretraining_val_dataloader, 
                        device       = device)

e1.load("runs/PSM_basic-pretraining-2021_09_25-15_48_12/model_manual_save.pt")

output = e1.eval_model(test_dataloader)
print("Mean 3PE: {}%, Mean SmoothL1 loss: {}".format(round((1-output["accuracy"])*100,2), 
                                                     round(output["loss"], 4)))

/anaconda3/envs/pytorch/lib/python3.9/site-packages/torch/nn/functional.py:3498: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/anaconda3/envs/pytorch/lib/python3.9/site-packages/torch/nn/functional.py:3620: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(
/otherssd/labvision_project/cansu/labvision_stereo_depth_estimation_V02/models/psmnet_basic.py:193: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  cost = Variable(torch.zeros((refimg_fea.size()[0], refimg_fea.size()[1]*2, self.maxdisp//4,  refimg_fea.size()[2],  refimg_fea.size()[3])).zero_(), volatile= not self.training).cuda()
/anaconda3/envs/pytorch/lib/pyth

Mean 3PE: 9.39%, Mean SmoothL1 loss: 1.271399974822998


## Finetuned Models

### Full PSMNet

#### lr=1e-4, no scheduler

In [15]:
model = PSMNet(192).to(device)

e1 = t_utils.Experiment(model        = model,
                        criterion    = nn.SmoothL1Loss(), 
                        scheduler    = None,
                        optimizer    = torch.optim.Adam(model.parameters(), lr=1e-4), 
                        train_loader = finetuning_train_dataloader, 
                        val_loader   = finetuning_val_dataloader, 
                        device       = device)

e1.load("runs/PSM-finetuning-2021_09_25-01_29_49_lr1e-4/model_manual_save.pt")

output = e1.eval_model(test_dataloader)
print("Mean 3PE: {}%, Mean SmoothL1 loss: {}".format(round((1-output["accuracy"])*100,2), 
                                                     round(output["loss"], 4)))

Mean 3PE: 3.24%, Mean SmoothL1 loss: 0.5654000043869019


#### lr=1e-5, no scheduler

In [16]:
model = PSMNet(192).to(device)

e1 = t_utils.Experiment(model        = model,
                        criterion    = nn.SmoothL1Loss(), 
                        scheduler    = None,
                        optimizer    = torch.optim.Adam(model.parameters(), lr=1e-5), 
                        train_loader = finetuning_train_dataloader, 
                        val_loader   = finetuning_val_dataloader, 
                        device       = device)

e1.load("runs/PSM-finetuning-2021_09_25-04_32_59_lr1e-5/model_manual_save.pt")

output = e1.eval_model(test_dataloader)
print("Mean 3PE: {}%, Mean SmoothL1 loss: {}".format(round((1-output["accuracy"])*100,2), 
                                                     round(output["loss"], 4)))

Mean 3PE: 3.32%, Mean SmoothL1 loss: 0.5867999792098999


#### lr=1e-3, with scheduler

In [17]:
model = PSMNet(192).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

e1 = t_utils.Experiment(model        = model,
                        criterion    = nn.SmoothL1Loss(), 
                        scheduler    = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.75),
                        optimizer    = optimizer, 
                        train_loader = finetuning_train_dataloader, 
                        val_loader   = finetuning_val_dataloader, 
                        device       = device)

e1.load("runs/PSM-finetuning-2021_09_28-00_46_28_scheduler-lr1e-3/model_manual_save.pt")

output = e1.eval_model(test_dataloader)
print("Mean 3PE: {}%, Mean SmoothL1 loss: {}".format(round((1-output["accuracy"])*100,2), 
                                                     round(output["loss"], 4)))

Mean 3PE: 3.77%, Mean SmoothL1 loss: 0.6287000179290771


#### lr=1e-4, with scheduler

In [18]:
model = PSMNet(192).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

e1 = t_utils.Experiment(model        = model,
                        criterion    = nn.SmoothL1Loss(), 
                        scheduler    = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.75),
                        optimizer    = optimizer, 
                        train_loader = finetuning_train_dataloader, 
                        val_loader   = finetuning_val_dataloader, 
                        device       = device)

e1.load("runs/PSM-finetuning-2021_09_26-18_02_34_scheduler_lr1e-4/model_manual_save.pt")

output = e1.eval_model(test_dataloader)
print("Mean 3PE: {}%, Mean SmoothL1 loss: {}".format(round((1-output["accuracy"])*100,2), 
                                                     round(output["loss"], 4)))

Mean 3PE: 3.16%, Mean SmoothL1 loss: 0.5472999811172485


#### lr=1e-5, with scheduler

In [19]:
model = PSMNet(192).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

e1 = t_utils.Experiment(model        = model,
                        criterion    = nn.SmoothL1Loss(), 
                        scheduler    = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.75),
                        optimizer    = optimizer, 
                        train_loader = finetuning_train_dataloader, 
                        val_loader   = finetuning_val_dataloader,
                        device       = device)

e1.load("runs/PSM-finetuning-2021_09_26-21_32_49_scheduler_lr1e-5/model_manual_save.pt")

output = e1.eval_model(test_dataloader)
print("Mean 3PE: {}%, Mean SmoothL1 loss: {}".format(round((1-output["accuracy"])*100,2), 
                                                     round(output["loss"], 4)))

Mean 3PE: 4.51%, Mean SmoothL1 loss: 0.6977999806404114


#### Strided Cost Volume, lr=1e-4

In [20]:
costvolume_slide = 192//2
costvolume_stride = 2
model = PSMNet(192, costvolume_slide=costvolume_slide, costvolume_stride=costvolume_stride).to(device)

e1 = t_utils.Experiment(model        = model,
                        criterion    = nn.SmoothL1Loss(), 
                        scheduler    = None,
                        optimizer    = torch.optim.Adam(model.parameters(), lr=1e-4),
                        train_loader = finetuning_train_dataloader,
                        val_loader   = finetuning_val_dataloader,
                        device       = device)

e1.load("runs/PSM-finetuning-2021_09_25-17_03_04_stride2_lr1e-4/model_manual_save.pt")

output = e1.eval_model(test_dataloader)
print("Mean 3PE: {}%, Mean SmoothL1 loss: {}".format(round((1-output["accuracy"])*100,2), 
                                                     round(output["loss"], 4)))

Mean 3PE: 4.51%, Mean SmoothL1 loss: 0.7081000208854675


#### With ResNet50, lr=1e-4

In [21]:
model = PSMNet(192).to(device)
model.feature_extraction = FeatureExtractionPlusResNet50().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
e1 = t_utils.Experiment(model        = model,
                        criterion    = nn.SmoothL1Loss(), 
                        scheduler    = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.75),
                        optimizer    = optimizer, 
                        train_loader = finetuning_train_dataloader, 
                        val_loader   = finetuning_val_dataloader,
                        device       = device)

e1.load("runs/PSM-Resnet50-finetuning-2021_09_29-21_52_52/model_manual_save.pt")

output = e1.eval_model(test_dataloader)
print("Mean 3PE: {}%, Mean SmoothL1 loss: {}".format(round((1-output["accuracy"])*100,2), 
                                                     round(output["loss"], 4)))

Mean 3PE: 3.41%, Mean SmoothL1 loss: 0.6036999821662903


### GCNet

#### lr=1e-3, no scheduler

In [22]:
model = GCnet(192).to(device)

e1 = t_utils.Experiment(model        = model,
                        criterion    = nn.SmoothL1Loss(), 
                        scheduler    = None,
                        optimizer    = torch.optim.Adam(model.parameters(), lr=1e-3), 
                        train_loader = finetuning_train_dataloader, 
                        val_loader   = finetuning_val_dataloader, 
                        device       = device)

e1.load("runs/GCnet-finetuning-2021_09_28-00_17_53_lr1e-3/model_manual_save.pt")

output = e1.eval_model(test_dataloader)
print("Mean 3PE: {}%, Mean SmoothL1 loss: {}".format(round((1-output["accuracy"])*100,2), 
                                                     round(output["loss"], 4)))

Mean 3PE: 4.41%, Mean SmoothL1 loss: 0.7685999870300293


#### lr=1e-4, no scheduler

In [23]:
model = GCnet(192).to(device)

e1 = t_utils.Experiment(model        = model,
                        criterion    = nn.SmoothL1Loss(), 
                        scheduler    = None,
                        optimizer    = torch.optim.Adam(model.parameters(), lr=1e-4), 
                        train_loader = finetuning_train_dataloader, 
                        val_loader   = finetuning_val_dataloader, 
                        device       = device)

e1.load("runs/GCnet-finetuning-2021_09_27-19_30_42_lr1e-4/model_manual_save.pt")

output = e1.eval_model(test_dataloader)
print("Mean 3PE: {}%, Mean SmoothL1 loss: {}".format(round((1-output["accuracy"])*100,2), 
                                                     round(output["loss"], 4)))

Mean 3PE: 4.41%, Mean SmoothL1 loss: 0.73580002784729


#### lr=1e-4, with scheduler

In [24]:
model = GCnet(192).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

e1 = t_utils.Experiment(model        = model,
                        criterion    = nn.SmoothL1Loss(), 
                        scheduler    = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.75),
                        optimizer    = optimizer, 
                        train_loader = finetuning_train_dataloader, 
                        val_loader   = finetuning_val_dataloader, 
                        device       = device)

e1.load("runs/GCnet-finetuning-2021_09_27-19_41_40_scheduler_lr1e-4/model_manual_save.pt")

output = e1.eval_model(test_dataloader)
print("Mean 3PE: {}%, Mean SmoothL1 loss: {}".format(round((1-output["accuracy"])*100,2), 
                                                     round(output["loss"], 4)))

Mean 3PE: 4.62%, Mean SmoothL1 loss: 0.8087000250816345


### Basic PSMNet

In [25]:
model = PSMNet_basic(192).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

e1 = t_utils.Experiment(model        = model,
                        criterion    = nn.SmoothL1Loss(), 
                        scheduler    = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.75),
                        optimizer    = optimizer, 
                        train_loader = finetuning_train_dataloader, 
                        val_loader   = finetuning_val_dataloader, 
                        device       = device)

e1.load("runs/PSM_basic-finetuning-2021_09_28-23_48_34/model_manual_save.pt")

output = e1.eval_model(test_dataloader)
print("Mean 3PE: {}%, Mean SmoothL1 loss: {}".format(round((1-output["accuracy"])*100,2), 
                                                     round(output["loss"], 4)))

Mean 3PE: 3.48%, Mean SmoothL1 loss: 0.5889000296592712


## Without Pretraining

### Full PSMNet

#### lr=1e-4, no scheduler

In [26]:
model = PSMNet(192).to(device)

e1 = t_utils.Experiment(model        = model,
                        criterion    = nn.SmoothL1Loss(), 
                        scheduler    = None,
                        optimizer    = torch.optim.Adam(model.parameters(), lr=1e-4), 
                        train_loader = finetuning_train_dataloader, 
                        val_loader   = finetuning_val_dataloader, 
                        device       = device)

e1.load("runs/PSMNet-training-2021_09_26-05_14_10_lr1e-4/model_manual_save.pt")

output = e1.eval_model(test_dataloader)
print("Mean 3PE: {}%, Mean SmoothL1 loss: {}".format(round((1-output["accuracy"])*100,2), 
                                                     round(output["loss"], 4)))

Mean 3PE: 4.91%, Mean SmoothL1 loss: 0.7924000024795532


#### lr=1e-4, with scheduler

In [27]:
model = PSMNet(192).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

e1 = t_utils.Experiment(model        = model,
                        criterion    = nn.SmoothL1Loss(), 
                        scheduler    = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.75),
                        optimizer    = optimizer, 
                        train_loader = finetuning_train_dataloader, 
                        val_loader   = finetuning_val_dataloader, 
                        device       = device)

e1.load("runs/PSMNet-training-2021_09_27-05_52_04_scheduler_lr1e-4/checkpoints/ep_20000.pt")

output = e1.eval_model(test_dataloader)
print("Mean 3PE: {}%, Mean SmoothL1 loss: {}".format(round((1-output["accuracy"])*100,2), 
                                                     round(output["loss"], 4)))

Mean 3PE: 5.27%, Mean SmoothL1 loss: 0.8180000185966492


#### With max disparity=96, lr=1e-4, with scheduler

In [28]:
model = PSMNet(96).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

e1 = t_utils.Experiment(model        = model,
                        criterion    = nn.SmoothL1Loss(), 
                        scheduler    = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.75),
                        optimizer    = optimizer, 
                        train_loader = finetuning_train_dataloader,
                        val_loader   = finetuning_val_dataloader,
                        device       = device)

e1.load("runs/PSMNet-training-2021_09_28-07_32_46_maxdisp96_scheduler_lr1e-4/model_manual_save.pt")

output = e1.eval_model(test_dataloader)
print("Mean 3PE: {}%, Mean SmoothL1 loss: {}".format(round((1-output["accuracy"])*100,2), 
                                                     round(output["loss"], 4)))

Mean 3PE: 8.24%, Mean SmoothL1 loss: 1.0264999866485596
